# AIモデル訓練サンプル
PythonでEng-Fraサンプルデータを用いて，FrenchからEnglishへ翻訳するモデルを訓練する。
本モデルでは、単語単位ではなく、文字単位で分割し、学習します。

## プログラム説明

model.py

内部で，pytorchのtransformerライブラリを呼び出している。各パラメタの詳細は [Pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Transformer.html) を参照のこと。

- emb_size    
  単語のエンベッドサイズ
- nhead    
  transformerのMultiHeadAttentionのヘッド数
- ffn_hid_dim       
  FeedForwardNeuralNetworkの次元数
- batch_size     
  ミニバッチサイズ。メモリが足りないときや計算速度を早めたいときにはこのサイズを変更する
- num_encoder_layers    
  エンコーダ内のサブエンコーダ層の数 
- num_decoder_layers    
  デコーダ内のサブデコーダ層の数
- lr   
  学習率
- dropout    
  ドロップアウトの割合，1=100%
- num_epochs    
  学習用データを何周学習するか
- device    
  cuda: Cudaが使えるマシンではこれを選択
  mps: Apple Silicornが使えるマシンではこれを選択
  cpu: CPUで計算
- earlystop_patient    
  num_epochs以下でも，開発用データで，Lossが下がらなくなった回数がearlystop_patientより大きくなると，計算を終了させる
- output_dir    
  学習したモデルを格納するディレクトリ。ディレクトリには checkpoint_xxx.pt（xxxはepoch数）とcheckpoint_best.ptが作成され，valid lossが最も小さくなったepoch回のモデルをcheckpoint_best.ptとして保存する
- tensorboard_logdir    
  tensorboard のログを格納するディレクトリ。学習結果などを視覚化して表示できる。 tensorboard --logdir tensorboard_logdir で起動し，http://localhost:6006でアクセスすると表示される
- prefix    
  jsonl形式の訓練データ及び開発データのprefix
- source_lang    
  jsonl形式の訓練データ及び開発データでの，翻訳元となるデータにつけるキー
- target_lang     
  jsonl形式の訓練データ及び開発データでの，翻訳先となるデータにつけるキー

In [1]:
# Pythonで訓練をする
# deviceはcuda or cpu or mps
#  mps: apple silicon
# out of memoryが発生した際には、batch_sizeを減らす


# 途中から計算するときには，modelディレクトリに checkpoint_xxx.pt(xxxは計算済みのepoch数)とcheckpoint_best.pt が存在すること

!python ./model.py \
  --emb_size 1024 \
  --nhead 8 \
  --ffn_hid_dim 2048 \
  --batch_size 32 \
  --num_encoder_layers 12 \
  --num_decoder_layers 12 \
  --lr 0.00002 \
  --dropout 0.3 \
  --num_epochs 100 \
  --device cuda \
  --earlystop_patient 3 \
  --output_dir model \
  --tensorboard_logdir logs \
  --prefix translation \
  --source_lang fra \
  --target_lang eng \
  --train_file ../dataset/train.jsonl \
  --valid_file ../dataset/val.jsonl

num_epochs:100
Traceback (most recent call last):
  File "/home/analysis01/src/kanjikana-model/sample/training/./model.py", line 486, in <module>
    main()
  File "/home/analysis01/src/kanjikana-model/sample/training/./model.py", line 482, in main
    KanjiKanaTransformer(args).train()
  File "/home/analysis01/src/kanjikana-model/sample/training/./model.py", line 427, in train
    train_loss = self.train_epoch(train_iter, transformer, optimizer, loss_fn)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/analysis01/src/kanjikana-model/sample/training/./model.py", line 242, in train_epoch
    logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/analysis01/src/kanjikana-model/venv/lib/python3.11/site-packages/torch/nn/modules/module.py", line 1736, in _wrapped_call_impl
    r

In [2]:
# javaのDJLで使えるようにモデルファイルを変換する

In [3]:
!python convert.py  \
    --model_file=model/checkpoint_best.pt \
    --model_script=model/script.pt \
    --encoder=model/encoder.pt \
    --decoder=model/decoder.pt \
    --positional_encoding=model/positional_encoding.pt \
    --generator=model/generator.pt \
    --src_tok_emb=model/src_tok_emb.pt \
    --tgt_tok_emb=model/tgt_tok_emb.pt \
    --vocab_src=model/vocab_src.txt \
    --vocab_tgt=model/vocab_tgt.txt \
    --params=model/params.json \
    --device=cpu


Traceback (most recent call last):
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/sample/training/convert.py", line 75, in <module>
    main()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/sample/training/convert.py", line 70, in main
    KanjiKanaTransformerScripted(args).convert()
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/sample/training/convert.py", line 19, in convert
    best_checkpoint = torch.load(self.args.model_file, map_location=torch.device(self.args.device))
                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/venv/lib/python3.12/site-packages/torch/serialization.py", line 1319, in load
    with _open_file_like(f, "rb") as opened_file:
         ^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/utsubo-katsuhiko/Documents/GitHub/kanjikana-model/venv/lib/python3.12/site-packages/torch/serialization.py", line 659, in _